### SoccerNet Object Tracking
#### 02456 Deep learning 2022

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import os
import time
import yaml
import re
import seaborn as sn
from IPython.display import Video, Image
from glob import glob
from io import TextIOWrapper
import collections
import zipfile
import csv
import cv2

##### Data downloader

In [2]:
# import SoccerNet
# from SoccerNet.Downloader import SoccerNetDownloader
# mySoccerNetDownloader=SoccerNetDownloader(LocalDirectory="path/to/SoccerNet")
# mySoccerNetDownloader.downloadDataTask(task="tracking", split=["train", "test", "challenge"])

##### Function to visualize boxes

In [3]:
def get_np_array_from_tar_object(tar_extractfl):
     '''converts a buffer from a tar file in np.array'''
     return np.asarray(bytearray(tar_extractfl.read()), dtype=np.uint8)

def collect(fn):
    archive = zipfile.ZipFile(fn, 'r')
    annotated_images = []

    annotation_files = []
    for f in archive.namelist():
        if "det/det.txt" in f:
            annotation_files.append(f)

    image_annotations = collections.defaultdict(list)

    for af in annotation_files:
        st, sample,_,__ = af.split("/")
        imgpath = "%s/%s/img1/%06d.jpg"
        for row in csv.reader(TextIOWrapper(archive.open(af), 'utf-8') ):
            frame, _, x, y, w, h = [int(x) for x in row[:6]]
            ifn = imgpath%(st,sample, frame)
            image_annotations[ifn].append((x,y,w,h))

    for k in image_annotations.keys():
        img = cv2.imdecode(get_np_array_from_tar_object(archive.open(k)), 1 )
        for box in image_annotations[k]:
            x,y,w,h = box
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0), 2)

        img = cv2.resize(img, (0,0), fx=0.75, fy=0.75)
        cv2.imshow('image',img)
        cv2.waitKey(100)

In [4]:
# # Visualize
# train = collect("path/to/SoccerNet/tracking/train.zip")
# cv2.destroyAllWindows()

##### Test YOLOv7 with SoccerNet

In [3]:
yolo_base = 'yolov7'
sn_track_base = 'path/to/SoccerNet/tracking'

# ４ types of labels are used.
labels = ['ball', 'player', 'referee', 'goalkeepers']
label_dict = {'ball': 0, 'player': 1, 'referee': 2, 'goalkeeper': 3, 'goalkeepers': 3}

In [13]:
# predict!
check_images = f'test_imgs'
weight_file = 'yolov7-w6_training.pt'
# weight_file = 'yolov7.pt'
!cd $yolo_base && python detect.py \
    --weights $weight_file \
    --source  $check_images \
    --img-size 1280 \
    --project dfl

Namespace(weights=['yolov7.pt'], source='test_imgs', img_size=1280, conf_thres=0.25, iou_thres=0.45, device='', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='dfl', name='exp', exist_ok=False, no_trace=False)
Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block

YOLOR  2022-11-14 torch 1.11.0+cpu CPU

Model Summary: 306 layers, 36905341 parameters, 6652669 gradients
C:\ProgramData\Anaconda3\lib\site-packages\torch\functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]



 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

22 persons, Done. (2355.3ms) Inference, (0.0ms) NMS
 The image with the result is saved in: dfl\exp2\000001.jpg
21 persons, Done. (2194.5ms) Inference, (15.6ms) NMS
 The image with the result is saved in: dfl\exp2\000002.jpg
21 persons, Done. (2188.0ms) Inference, (15.6ms) NMS
 The image with the result is saved in: dfl\exp2\000003.jpg
21 persons, Done. (2275.3ms) Inference, (0.0ms) NMS
 The image with the result is saved in: dfl\exp2\000004.jpg
20 persons, Done. (2813.6ms) Inference, (15.6ms) NMS
 The image with the result is saved in: dfl\exp2\000005.jpg
19 persons, Done. (2573.0ms) Inference, (15.6ms) NMS
 The image with the result is saved in: dfl\exp2\000006.jpg
20 persons, Done. (2754.1ms) Inference, (0.0ms) NMS
 The image with the result is saved in: dfl\exp2\000007.jpg
20 persons, Done. (2766.8ms) Inference, (0.0ms) NMS
 The image with the result is saved in: dfl\exp2\000008.jpg
19 persons, Do

In [ ]:
# Check some results. 
display(Image(f'{yolo_base}/dfl/exp9/000200.jpg'))
display(Image(f'{yolo_base}/dfl/exp9/000400.jpg'))
display(Image(f'{yolo_base}/dfl/exp9/000600.jpg'))